In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_excel("датасет 15072022.xlsx")
df.head()

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ
2,2112 Продукция,8415900009; 8415900009; 8415900009; 8415900009...,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Клапаны противопожарные нормально открытые, кл...",Клапаны противопожарные марки КПВ.01(EI60) пря...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu
3,50677 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы крышные радиальные дымоудаления с ...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu
4,51009 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы радиальные дымоудаления типов ВР-8...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu


In [2]:
df_test1 = df[["Группа продукции", "Технические регламенты", "Коды ТН ВЭД ЕАЭС", "Общее наименование продукции"]].copy()
df_test1.head()

,Группа продукции,Технические регламенты,Коды ТН ВЭД ЕАЭС,Общее наименование продукции
0,Узлы пересечения противопожарных преград кабел...,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,4016999708,Проходки кабельные модульные универсальные огн...
1,"Противопожарные окна, двери, двери шахт лифтов...",ТР ЕАЭС 043/2017 О требованиях к средствам обе...,7308909809,Ворота противопожарные металлические распашные...
2,"Клапаны противопожарные нормально открытые, кл...",ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8415900009; 8415900009; 8415900009; 8415900009...,Клапаны противопожарные марки КПВ.01(EI60) пря...
3,Вытяжные вентиляторы,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8414594000,Вентиляторы крышные радиальные дымоудаления с ...
4,Вытяжные вентиляторы,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8414594000,Вентиляторы радиальные дымоудаления типов ВР-8...


In [9]:
with open("F:\Hackaton\ТН ВЭД\TNVED\TNVED1.Txt", encoding='cp866') as fout_tnved1:
    dict_common = dict()
    fout_tnved1.readline()
    for line in fout_tnved1.readlines():
        line = line.replace('\xa0', '')
        ls = line.split('|')
        ls.pop(-1)
        dict_common[ls[0]] = ls[1]
    for i in dict_common.items():
        print(i, "\n")

('01', 'ЖИВЫЕ ЖИВОТНЫЕ; ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ') 

('02', 'ПРОДУКТЫ РАСТИТЕЛЬНОГО ПРОИСХОЖДЕНИЯ') 

('03', 'ЖИРЫ И МАСЛА ЖИВОТНОГО, РАСТИТЕЛЬНОГО ИЛИ МИКРОБИОЛОГИЧЕСКОГО ПРОИСХОЖДЕНИЯ И ПРОДУКТЫ ИХ РАСЩЕПЛЕНИЯ; ГОТОВЫЕ ПИЩЕВЫЕ ЖИРЫ; ВОСКИ ЖИВОТНОГО ИЛИ РАСТИТЕЛЬНОГО ПРОИСХОЖДЕНИЯ') 

('04', 'ГОТОВЫЕ ПИЩЕВЫЕ ПРОДУКТЫ; АЛКОГОЛЬНЫЕ И БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ И УКСУС; ТАБАК И ПРОМЫШЛЕННЫЕ ЗАМЕНИТЕЛИ ТАБАКА; ПРОДУКЦИЯ, СОДЕРЖАЩАЯ ИЛИ НЕ СОДЕРЖАЩАЯ НИКОТИН, ПРЕДНАЗНАЧЕННАЯ ДЛЯ ВДЫХАНИЯ БЕЗ ГОРЕНИЯ; ПРОЧАЯ ПРОДУКЦИЯ, СОДЕРЖАЩАЯ НИКОТИН И ПРЕДНАЗНАЧЕННАЯ ДЛЯ ПОСТУПЛЕНИЯ НИКОТИНА В ОРГАНИЗМ ЧЕЛОВЕКА') 

('05', 'МИНЕРАЛЬНЫЕ ПРОДУКТЫ') 

('06', 'ПРОДУКЦИЯ ХИМИЧЕСКОЙ И СВЯЗАННЫХ С НЕЙ ОТРАСЛЕЙ ПРОМЫШЛЕННОСТИ') 

('07', 'ПЛАСТМАССЫ И ИЗДЕЛИЯ ИЗ НИХ; КАУЧУК, РЕЗИНА И ИЗДЕЛИЯ ИЗ НИХ') 

('08', 'НЕОБРАБОТАННЫЕ ШКУРЫ, ВЫДЕЛАННАЯ КОЖА, НАТУРАЛЬНЫЙ МЕХ И ИЗДЕЛИЯ ИЗ НИХ; ШОРНО-СЕДЕЛЬНЫЕ ИЗДЕЛИЯ И УПРЯЖЬ; ДОРОЖНЫЕ ПРИНАДЛЕЖНОСТИ, СУМКИ И АНАЛОГИЧНЫЕ ИМ ТОВАРЫ; ИЗДЕЛИЯ ИЗ ВНУТРЕННИХ ОРГАНОВ ЖИВОТНЫХ

In [8]:
with open("F:\Hackaton\ТН ВЭД\TNVED\TNVED2.Txt", encoding='cp866') as fout_tnved2:
    dict_group = dict()
    fout_tnved2.readline()
    for line in fout_tnved2.readlines():
        line = line.replace('\xa0', '')
        ls = line.split('|')
        ls.pop(-1)
        dict_group[ls[0] + ls[1]] = ls[2]
    for i in dict_group.items():
        print(i, "\n")

('0101', 'ЖИВЫЕ ЖИВОТНЫЕ') 

('0102', 'МЯСО И ПИЩЕВЫЕ МЯСНЫЕ СУБПРОДУКТЫ') 

('0103', 'РЫБА И РАКООБРАЗНЫЕ, МОЛЛЮСКИ И ПРОЧИЕ ВОДНЫЕ БЕСПОЗВОНОЧНЫЕ') 

('0104', 'МОЛОЧНАЯ ПРОДУКЦИЯ; ЯЙЦА ПТИЦ; МЕД НАТУРАЛЬНЫЙ; ПИЩЕВЫЕ ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ, В ДРУГОМ МЕСТЕ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ') 

('0105', 'ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ, В ДРУГОМ МЕСТЕ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ') 

('0206', 'ЖИВЫЕ ДЕРЕВЬЯ И ДРУГИЕ РАСТЕНИЯ; ЛУКОВИЦЫ, КОРНИ И ПРОЧИЕ АНАЛОГИЧНЫЕ ЧАСТИ РАСТЕНИЙ; СРЕЗАННЫЕ ЦВЕТЫ И ДЕКОРАТИВНАЯ ЗЕЛЕНЬ') 

('0207', 'ОВОЩИ И НЕКОТОРЫЕ СЪЕДОБНЫЕ КОРНЕПЛОДЫ И КЛУБНЕПЛОДЫ') 

('0208', 'СЪЕДОБНЫЕ ФРУКТЫ И ОРЕХИ; КОЖУРА ЦИТРУСОВЫХ ПЛОДОВ ИЛИ КОРКИ ДЫНЬ') 

('0209', 'КОФЕ, ЧАЙ, МАТЕ, ИЛИ ПАРАГВАЙСКИЙ ЧАЙ, И ПРЯНОСТИ') 

('0210', 'ЗЛАКИ') 

('0211', 'ПРОДУКЦИЯ МУКОМОЛЬНО-КРУПЯНОЙ ПРОМЫШЛЕННОСТИ; СОЛОД; КРАХМАЛЫ; ИНУЛИН; ПШЕНИЧНАЯ КЛЕЙКОВИНА') 

('0212', 'МАСЛИЧНЫЕ СЕМЕНА И ПЛОДЫ; ПРОЧИЕ СЕМЕНА, ПЛОДЫ И ЗЕРНО; ЛЕКАРСТВЕННЫЕ РАСТЕНИЯ И РАСТЕНИЯ ДЛЯ ТЕХНИЧЕСКИХ ЦЕЛЕЙ